# Crop : best.pt 를 이용해서 해당 class 의 옷만 crop

In [ ]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from ultralytics import YOLO

conf_threshold = 0.5   # 감지 신뢰도 기준 (예: 0.3, 0.5, 0.7 ...)
iou_threshold  = 0.45  # NMS IoU 기준 (필요시 조절)
output_dir     = "crop_29cm"

# YOLO 모델 로드
model = YOLO("best.pt")

# CSV 읽기 (예: 컬럼 ["상품코드","대분류","이미지URL"])
# df = pd.read_csv("product_url.csv")

# 결과 저장 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# 대분류 → YOLO 라벨 매핑
label_map = {
    "상의": "top",
    "스커트": "skirt",
    "원피스": "dress",
    "바지": "pants",
}

# 이미지 다운로드 및 YOLO 탐지 후 크롭
for idx, row in df.iterrows():
    product_id = str(row["상품코드"])
    category_kr = str(row["대분류"]).strip()
    url = row["이미지URL"]

    target_label = label_map.get(category_kr)
    if target_label is None:
        print(f"[스킵] 지원하지 않는 대분류: {category_kr}")
        continue

    try:
        # 이미지 다운로드
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        img = Image.open(BytesIO(resp.content)).convert("RGB")

        # YOLO 추론 (conf/iou를 파라미터로 전달)
        # ※ 아래 conf로 1차 필터가 되고, 그래도 안전하게 후처리에서도 한 번 더 체크
        results = model(img, conf=conf_threshold, iou=iou_threshold, verbose=False)

        # 대상 라벨만 모아서, 그 중 conf가 가장 높은 것 하나만 선택
        best = None  # (conf_float, box_tensor)
        for r in results:
            # r.boxes.conf, r.boxes.cls, r.boxes.xyxy 는 길이가 동일
            for box, cls, conf in zip(r.boxes.xyxy, r.boxes.cls, r.boxes.conf):
                label = model.names[int(cls)].lower()
                conf_val = float(conf.item()) if hasattr(conf, "item") else float(conf)
                if label == target_label and conf_val >= conf_threshold:
                    if (best is None) or (conf_val > best[0]):
                        best = (conf_val, box)

        if best is None:
            print(f"[스킵] {product_id} → '{target_label}' 감지 없음 (conf>={conf_threshold})")
            continue

        # 최고 conf 박스만 저장
        _, best_box = best
        x1, y1, x2, y2 = map(int, best_box)
        cropped_img = img.crop((x1, y1, x2, y2))
        save_path = os.path.join(output_dir, f"{target_label}_{product_id}.jpg")
        cropped_img.save(save_path)
        print(f"[저장 완료] {save_path} (conf={best[0]:.2f})")

    except Exception as e:
        print(f"[오류] {product_id} / {url} → {e}")
